In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
!ls 'gdrive'/'My Drive'/'RogerHeederer'/'NLP_KR'

2.NlpPrep_heederer	      6.3.preprocess.ipynb  7.PretrainMethod_heederer
4.TextClassfication_heederer  6.4.seq2seq.ipynb     data
4.TEXT_CLASSIFICATION	      6.CHATBOT		    preprocess.py
6.2.EDA.ipynb		      6.CHATBOT_heederer


In [3]:
!ls 'gdrive/My Drive/RogerHeederer/NLP_KR/6.CHATBOT_heederer'

6_1.chatbot_EDA.ipynb	      6_3.chatbot_seq2seq.ipynb  preprocess.py
6_2.chatbot_preprocess.ipynb  data_in			 __pycache__


In [4]:
# 훈련 데이터, 데이터 로더, 파이썬 파일 등을 불러오기 위해 경로 설정
import sys
sys.path.append('gdrive/My Drive/RogerHeederer/NLP_KR/6.CHATBOT_heederer')

In [5]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 7.7MB/s 
     |████████████████████████████████| 460kB 42.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [6]:
%load_ext autoreload
%autoreload 2

In [46]:
%reload_ext autoreload
from preprocess import *

from importlib import reload  
import foo

while True:
    # Do some things.
    if is_changed(foo):
        foo = reload(foo)

스택 오버플로에 이 방법이 있지만 나는 import *(전체)를 해야해서 적용 어케 할지 모르겠음
암튼 내 케이스랑은 맞지 않아

근데 위 방법으로 하니, .py 파일 변경 후 몇초 흘려보내고, reload_ext autoreload와
from preprocess import * 한번 다시 실행시키고 하니까

.py 파일 변경된게 반영된다. 

처음에는 import 만 다시 겁나 해봤는데 도통 변경 사항이 반영되지 않았다. 
세션을 다 끊고 다시 연결해 처음부터 하면 되는데, 이걸 매번 할 수는 없는 일(난 google drive에서 .py 파일을 texteditor 연결 앱으로 바로 열어서 라이브로 바로 바로 수정한다)

구글 뒤져보니 I suspect what's happening is you have .py files stored in Google Drive, which you're importing in your notebook, and you want to see changes to the .py files reflected in your runtime, but they're not because python's import system is idempotent (an import statement is ignored if python thinks it's already loaded a module by that name, even if the underlying file has changed).

런 타이중에 변경된 .py의 내용들이 반영되지 않는다. 파이썬 임포트 시스템이 무시한다.

본 파일이 변경 되었더라도 Python이 이미 해당 이름으로 모듈을로드했다고 생각하면 import 문이 무시 됨)



In [9]:
BASE_PATH = 'gdrive/My Drive/RogerHeederer/NLP_KR/6.CHATBOT_heederer'

In [10]:
DATA_PATH = BASE_PATH + '/data_in/ChatBotData.csv'
VOCAB_PATH = BASE_PATH + '/data_in/vocabulary.txt'

In [11]:
#데이터 로드
inputs, outputs = load_data(DATA_PATH)

In [12]:
inputs[0:5]

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']

In [13]:
outputs[0:5]

['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.']

In [14]:
char2idx, idx2char, vocab_size = load_vocabulary(DATA_PATH, VOCAB_PATH, tokenize_as_morph=True)

위에 만들어진 단어사전을 통해 모델에 학습할 인덱스 데이터를 구성한다(아래처럼)

In [24]:
index_inputs, input_seq_len = enc_processing(inputs, char2idx, tokenize_as_morph=True)

100%|██████████| 11823/11823 [00:29<00:00, 406.26it/s]


In [35]:
index_outputs, output_seq_len = dec_output_processing(outputs, char2idx, tokenize_as_morph=True)

100%|██████████| 11823/11823 [00:38<00:00, 310.27it/s]


In [47]:
index_targets = dec_target_processing(outputs, char2idx, tokenize_as_morph=True)

100%|██████████| 11823/11823 [00:38<00:00, 304.37it/s]


자 인제 인덱스 데이터는 모두 구성했다.

이젠, 모델 학습할 때와 추론할 때 활용하기 위해 단어 사전과 특별 토큰들을 각각 정의해서 딕셔너리 객체에 담아둔다

In [49]:
data_configs = {}
data_configs['char2idx'] = char2idx
data_configs['idx2char'] = idx2char
data_configs['vocab_size'] = vocab_size
data_configs['pad_symbol'] = PAD
data_configs['std_symbol'] = STD
data_configs['end_symbol'] = END
data_configs['unk_symbol'] = UNK

인덱스 데이터와 단어사전을 구성한 딕셔너리 객체를 numpy, json 형식으로 저장

In [50]:
DATA_IN_PATH = BASE_PATH + '/data_in/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

np.save(open(DATA_IN_PATH + TRAIN_INPUTS, 'wb'), index_inputs)
np.save(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'wb'), index_outputs)
np.save(open(DATA_IN_PATH + TRAIN_TARGETS , 'wb'), index_targets)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'))

자 이제 모델에 학습시킬 데이터 준비는 마무리 되었다

In [51]:
char2idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<END>': 2,
 '<UNK>': 3,
 '났나': 4,
 '종교문제결국엔헤어지네': 5,
 '내릴수': 6,
 '밖': 7,
 '힘내라는': 8,
 '헤어지면': 9,
 '걸어가고있는데': 10,
 '차별': 11,
 '하구싶다': 12,
 '아프네요': 13,
 '걸릴지도': 14,
 '물어주시다니': 15,
 '간사귀다이별했습니다': 16,
 '비장': 17,
 '한다': 18,
 '별도': 19,
 '꽉찼어': 20,
 '썼길': 21,
 '말렸어': 22,
 '마을': 23,
 '썼어': 24,
 '해준게': 25,
 '편': 26,
 '듣고싶네요': 27,
 '행복해지세요': 28,
 '들이말': 29,
 '모르는것도': 30,
 '볼': 31,
 '사는거': 32,
 '중요': 33,
 '내줄게요': 34,
 '40': 35,
 '힘들': 36,
 '반해서': 37,
 '안타깝지만': 38,
 '빌려줘': 39,
 '받는다고': 40,
 '지날수있을거예요': 41,
 '겁나': 42,
 '놓는것도': 43,
 '분명히': 44,
 '가배': 45,
 '우울해': 46,
 '치킨': 47,
 '뿌염': 48,
 '봄날': 49,
 '지친다': 50,
 '무서워요': 51,
 '남어': 52,
 '시세': 53,
 '편한거': 54,
 '해보신': 55,
 '보내는거': 56,
 '꺼진': 57,
 '해보긴': 58,
 '명치': 59,
 '생강차': 60,
 '목표': 61,
 '않으셨나': 62,
 '공적': 63,
 '붙잡으세요': 64,
 '하고말하는': 65,
 '힘겨': 66,
 '종일': 67,
 '잘살수있을까': 68,
 '를': 69,
 '분야': 70,
 '신나게': 71,
 '씻고': 72,
 '줄어디': 73,
 '핸폰': 74,
 '줬어': 75,
 '되는거죠': 76,
 '이정': 77,
 '나를안': 78,
 '종': 79,
 '싫은것': 80,
 '재밌다고': 81,
 